In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import networkx as nx
import numpy as np
import cvxpy as cp
from tqdm import tqdm
from pathlib import Path

from src.load_data import (
    read_metadata_networks_tntp,
    read_graph_transport_networks_tntp,
    read_traffic_mat_transport_networks_tntp,
)

from src.models import SDModel, BeckmannModel, TwostageModel
from src.algs import subgd, ustm, frank_wolfe, cyclic, N_conjugate_frank_wolfe 
from src.cvxpy_solvers import get_max_traffic_mat_mul
from src.commons import Correspondences

import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 14})
%config InlineBackend.figure_format = 'retina'

%matplotlib inline

In [ ]:
networks_path = Path("./TransportationNetworks")

folder = "SiouxFalls"
net_name = "SiouxFalls_net"
traffic_mat_name = "SiouxFalls_trips"

# folder = "Anaheim"
# net_name = "Anaheim_net"
# traffic_mat_name = "Anaheim_trips"
    
# folder = "Barcelona"
# net_name = "Barcelona_net"
# traffic_mat_name = "Barcelona_trips"

    
net_file = networks_path / folder / f"{net_name}.tntp"
traffic_mat_file = networks_path / folder / f"{traffic_mat_name}.tntp"
graph, metadata = read_graph_transport_networks_tntp(net_file)
correspondences = read_traffic_mat_transport_networks_tntp(traffic_mat_file, metadata)
n = graph.number_of_nodes()

print(f"{graph.number_of_edges()=}, {graph.number_of_nodes()=}")

In [ ]:
beckmann_model = BeckmannModel(graph, correspondences)

eps = 1e-4
mean_bw = beckmann_model.graph.ep.capacities.a.mean()
mean_cost = beckmann_model.graph.ep.free_flow_times.a.mean()

# cost suboptimality <= eps * (average link cost * avg bandwidth * |E| \approx total cost when beta=1)
eps_abs = eps * mean_cost * mean_bw * graph.number_of_edges()

eps_cons_abs = eps * mean_bw 
# sum of capacity violation <= eps * average link capacity
print(eps_abs, eps_cons_abs)

# Beckmann

In [ ]:
times_e_ustm, flows_e_ustm, logs, optimal = ustm(beckmann_model, eps_abs, max_iter=2000, stop_by_crit=False)
dgap_ustm, cons_log_ustm, time_log_ustm = logs
print(len(dgap_ustm), "shortest paths calls")


In [ ]:
times_e_fw, flows_e_fw, logs, optimal = frank_wolfe(beckmann_model, eps_abs,  max_iter=7700, stop_by_crit=False)
dgap_fw, time_log_fw , primal_r_gap_fw = logs

In [ ]:
times_e_nfw, flows_e_nfw, logs, optimal = N_conjugate_frank_wolfe(beckmann_model, eps_abs,  max_iter=4000, stop_by_crit=False, cnt_conjugates=3 , linesearch=True)
dgap_nfw, time_log_nfw , primal_r_gap_nfw = logs

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(121)
dgap_ustm = np.abs(dgap_ustm)
dgap_fw = np.abs(dgap_fw)
dgap_nfw = np.abs(dgap_nfw)
plt.plot(time_log_ustm, dgap_ustm, c="C4", label="USTM")
plt.plot(time_log_fw, dgap_fw, c="C5", label="FW")
plt.plot(time_log_nfw, dgap_nfw, c="C3", label="NFW")
plt.axhline(y=np.ones(max(dgap_ustm.size, dgap_fw.size))[0] * eps_abs, linestyle= '--', label='eps_abs')
plt.yscale("log")
plt.title("abs(dgap)")
plt.legend()


# SD

In [ ]:
# %%time

# for Anaheim 18 min on my laptop (for np.ones matrix ~3 min, omg. maybe scaling impacts?). result \approx 0.529
# sd_max_traffic_mul = get_max_traffic_mat_mul(graph, correspondences.traffic_mat, solver=cp.SCS)
sd_max_traffic_mul = 0.529

# for SiouxFalls
# sd_max_traffic_mul = get_max_traffic_mat_mul(graph, correspondences.node_traffic_mat, solver=cp.ECOS, 
#                                              max_iters=1000, verbose=False)

print(f"{sd_max_traffic_mul=},")

corrs = Correspondences(traffic_mat=correspondences.traffic_mat * sd_max_traffic_mul * 0.9,
                        node_traffic_mat=correspondences.node_traffic_mat * sd_max_traffic_mul * 0.9,
                        sources=correspondences.sources,
                        targets=correspondences.targets)

sd_model = SDModel(graph, corrs)
# model = SDModel(graph, traffic_mat * sd_max_traffic_mul * 1.1)  # you should see that constraints error does not converge

## cvxpy edge-based

## Ustm

In [ ]:
%%time

times_e_ustm, flows_e_ustm, logs, optimal = ustm(sd_model, eps_abs, eps_cons_abs,
                                                                         max_iter=1000, stop_by_crit=True)

dgap_ustm, cons_log_ustm, time_log_ustm = logs
print(len(dgap_ustm), "iters")


In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(121)
dgap_abs = np.abs(dgap_ustm)
plt.plot(dgap_abs, c="C4", label="USTM")
plt.plot(np.ones(dgap_abs.size) * eps_abs, label="eps_abs")
plt.yscale("log")
plt.ylim(None, dgap_abs.max() * 2)
plt.title("abs(dgap)")
plt.legend()

plt.subplot(122)
plt.plot(np.ones(len(cons_log_ustm)) * eps_cons_abs, label="eps_cons_abs")
plt.plot(cons_log_ustm, c="C4", label="USTM")
plt.title("constraint violation")
plt.yscale("log")
plt.legend()

In [ ]:
bws = sd_model.graph.ep.capacities.a
order = bws.argsort()

plt.plot(bws[order], label="capacity", c='C2')
# plt.plot(flows_cvx[order], label="cvxpy", c="C1")
plt.plot(flows_e_ustm[order], label="ustm", c="C4")

plt.legend()

# Subgd

In [ ]:
%%time

times_e_subgd, flows_e_subgd, logs, optimal = subgd(sd_model, R=10, eps_abs=eps_abs, 
                                                                   eps_cons_abs=eps_cons_abs, max_iter=1000)

dgap_subgd, cons_log_subgd = logs

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(121)
dgap_abs = np.abs(dgap_subgd)
plt.plot(np.ones(dgap_abs.size) * eps_abs, label="eps_abs")
plt.plot(dgap_abs, c="C3", label="subgd")
plt.legend()
plt.yscale("log")

plt.title("abs(dgap)")
plt.subplot(122)
plt.plot(np.ones(len(cons_log_subgd)) * eps_cons_abs, label="eps_cons_abs")
plt.plot(cons_log_subgd, c="C3", label="subgd")
plt.legend()
plt.title("constraint violation l1 norm")
plt.yscale("log")


In [ ]:
bws = sd_model.graph.ep.capacities.a
order = bws.argsort()

plt.plot(bws[order], label="capacity", c='C2')
# plt.plot(flows_cvx[order], label="cvxpy", c="C1")
plt.plot(flows_e_subgd[order], label="subgd", c="C4")

plt.legend()

# Twostage

In [ ]:
traffic_mat = beckmann_model.correspondences.traffic_mat 
departures, arrivals = traffic_mat.sum(axis=1), traffic_mat.sum(axis=0) 
twostage_beckmann_model = TwostageModel(beckmann_model, departures=departures, arrivals=arrivals, gamma=0.1)

print(eps_abs, eps_cons_abs)

In [ ]:
times_e_cyclic, flows_e_cyclic, d_ij_cyclic, logs, optimal = cyclic(model=twostage_beckmann_model, 
                                                                eps_abs=eps_abs, 
                                                                traffic_assigment_eps_abs=eps_abs/10, 
                                                                traffic_assigment_max_iter=1000, 
                                                                max_iter=10, 
                                                                stop_by_crit=False)
dgap_cyclic, cons_log_cyclic, time_log_cyclic = logs

In [ ]:
times_ustm_2s, primal_var_ustm_2s, logs, optimal = ustm(twostage_beckmann_model, eps_abs=100, max_iter=1000, stop_by_crit=False)
dgap_ustm_2s, cons_log_ustm_2s, time_log_ustm_2s = logs
print(len(dgap_ustm_2s), "iters")

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(121)
plt.plot(time_log_cyclic, dgap_cyclic, c="C4", label="cyclic")
plt.scatter(time_log_cyclic, dgap_cyclic, c="C4")
plt.plot(time_log_ustm_2s, dgap_ustm_2s, c="C6", label="ustm")
plt.yscale("log")
plt.ylabel("dgap")
plt.xlabel("time")
plt.legend()

In [ ]:
print(np.linalg.norm(flows_e_cyclic - primal_var_ustm_2s[0]), np.linalg.norm(flows_e_cyclic))
print(np.linalg.norm(d_ij_cyclic - primal_var_ustm_2s[1]), np.linalg.norm(d_ij_cyclic))

In [ ]:
l, w = departures, arrivals

ustm_2s_flows, ustm_2s_corrs = primal_var_ustm_2s
w - ustm_2s_corrs.sum(axis=0)

In [ ]:
from src.admm_stuff import entr, AdmmOracle

In [ ]:
admm_oracle = AdmmOracle(twostage_beckmann_model.traffic_model, twostage_beckmann_model.gamma, l, w)

In [ ]:
import time
from src.cvxpy_solvers import admm_argmin_traffic, admm_argmin_flows

traffic_mat = beckmann_model.correspondences.traffic_mat
graph = beckmann_model.nx_graph
incidence_mat = nx.incidence_matrix(graph, oriented=True).todense()

d_ij = traffic_mat.copy()  # maybe init with zeros or 1-rank approx?
# d_ij = np.zeros(traffic_mat.shape)
rho = 0.2 

y = np.zeros(traffic_mat.shape)

start = time.time()
f_dist, d_dist, cons, dual_res = [], [], [], []
f_dual_feas, d_dual_feas = [], []
lam, mu = None, None

# TODO: reuse previous solution as starting point for subproblems, terminate by crit. 

for k in range(10):
    print("time passed:", round(time.time() - start), "s")
    
    flows_ei_ = admm_oracle.agd_argmin_flows(d_ij, y, rho, iters=5000, M0=10) 
    if k > 0:
        print("flows change norm", np.linalg.norm(flows_ei_ - flows_ei))
    flows_ei = flows_ei_

    d_ij_, y_salim = admm_oracle.salim_argmin_corrs(y_admm=y, flows_ei=flows_ei, rho=rho, mu=0.001, L=1, iters=10000) 
    print("corrs change norm", np.linalg.norm(d_ij - d_ij_))
    dual_res.append(rho * np.linalg.norm(admm_oracle.ATmul(admm_oracle.Bmul(d_ij - d_ij_))))
    d_ij = d_ij_
    
    Bd_plus_Af = admm_oracle.Amul(flows_ei) + admm_oracle.Bmul(d_ij)
    y += rho * Bd_plus_Af
    
    f_dist.append(np.linalg.norm(primal_var_ustm_2s[0] - flows_ei.sum(axis=1)))
    d_dist.append(np.linalg.norm(primal_var_ustm_2s[1] - d_ij))
    
    cons.append(np.linalg.norm(Bd_plus_Af))  # primal residual
    
    print("\n", k)
    print(f"flows dist={f_dist[-1]}, corrs dist={d_dist[-1]}")
    print(f"constraint={cons[-1]}", f"optimality={dual_res[-1]}", "\n")
    
plt.plot(f_dist, label="f dist")
plt.plot(d_dist, label="d dist")
plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
plt.plot(cons, label="cons")
plt.plot(dual_res, label="dual res")
plt.legend()
plt.yscale("log")


In [ ]:
x = admm_oracle.agd_argmin_flows(d_ij, y, rho, iters=1000, M0=100, plot_convergence=True)

In [ ]:
%%time 
print(rho)
x, y_salim = admm_oracle.salim_argmin_corrs(y_admm=y,
                                      flows_ei=flows_ei,
                                      rho=rho,
                                      mu=1e-3,
                                      L=1,
                                      iters=1000,
                                      plot_convergence=True)